In [5]:
import mne
import numpy as np
import antropy as ant
import glob
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
from tqdm import tqdm
import neurokit2 as nk

In [6]:
mne.set_log_level('WARNING')

In [10]:
paths1 = glob.glob('epochs_ar/*')
paths2 = glob.glob('epochs_ar/con??-day0-control_epo.fif')
paths3 = glob.glob('epochs_button_ar/*')

paths1.sort()
paths2.sort()
paths3.sort()

# Concatenate the sorted lists in the desired order
paths = paths1 + paths2 + paths3
paths = paths2


first_run = True
for path in tqdm(paths):
    
    components = re.split('[-_]', os.path.basename(path))
    sub = components[0][3:]
    day = components[1]
    condition = components[2]
    if components[3][:-1] == 'button#':
        button = components[3]
        jhana = components[4]
    else: 
        button = 'all'
        jhana = 'all'
    if day == "day0":
        sub = int(sub)+10
    


    
    #analysis
    epochs = mne.read_epochs(path)
    
    if len(epochs) < 10: 
        lempel_ziv_norm = np.nan
        permutation_entropy = np.nan
        spectral_entropy = np.nan
        sample_entropy = np.nan
        hjorth_mobility = np.nan
        hjorth_complexity = np.nan
        average_corr_all = np.nan
    
    else:                   
        picks = mne.pick_types(epochs.info, eeg=True, exclude='bads')
        epochs_good_eeg = epochs.pick(picks)
        ch_names = epochs_good_eeg.info['ch_names']
        data = epochs_good_eeg.get_data(copy=False)
        n_epochs, n_channels, _ = data.shape

        ###################
        ### Compute PSD ###
        ###################
        frequency_bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 12),
            'beta': (12, 30),
            'gamma': (30, 45)
        }

        band_powers = {band: [] for band in frequency_bands}
        psd = epochs.compute_psd(fmin=0.5, fmax=45, n_fft=256, method='welch')
        psds, freqs = psd.get_data(return_freqs=True)
        
        band_powers = {band: np.zeros((psds.shape[1],)) for band in frequency_bands}
        
        for band, (fmin, fmax) in frequency_bands.items():
            freq_mask = (freqs >= fmin) & (freqs < fmax)
            band_power = np.mean(psds[:, :, freq_mask], axis=(0, 2))  # Mean across epochs and frequencies
            band_powers[band] = band_power
        
            
        ###################
        ### Compute complexity measures ###
        ###################

    
        for channel, ch_name in enumerate(ch_names):
            lempel_ziv_norm = []
            permutation_entropy = []
            spectral_entropy = []
            sample_entropy = []
            hjorth_mobility = []
            hjorth_complexity = []
            average_corr_all = []
            nk_fsi = []
            nk_lle = []
            nk_sampen = []
            nk_pen = []
            nk_lzc = []

            for epoch in range(len(data)):
            
                time_series = data[epoch,channel,:]
                threshold = np.median(time_series)
                binary_sequence = (time_series > threshold).astype(int)
                binary_string = ''.join(binary_sequence.astype(str))
        
                lempel_ziv_norm.append(ant.lziv_complexity(binary_string, normalize=True))
                permutation_entropy.append(ant.perm_entropy(time_series, normalize=True))
                spectral_entropy.append(ant.spectral_entropy(time_series, sf=100, method='welch', normalize=True))
                sample_entropy.append(ant.sample_entropy(time_series))
                mobility, complexity = ant.hjorth_params(time_series)
                hjorth_mobility.append(mobility)
                hjorth_complexity.append(complexity)


                fsi, info = nk.fishershannon_information(time_series)
                lle, info = nk.complexity_lyapunov(time_series)
                sampen, parameters = nk.entropy_sample(time_series)
                pen, info = nk.entropy_permutation(time_series)
                lzc, info = nk.complexity_lempelziv(time_series)

                nk_fsi.append(fsi*10**4)
                nk_lle.append(lle*100)
                nk_sampen.append(sampen)
                nk_pen.append(pen)
                nk_lzc.append(lzc)

                
            lempel_ziv_norm = np.round(np.mean(lempel_ziv_norm), 4)
            permutation_entropy = np.round(np.mean(permutation_entropy), 4)
            spectral_entropy = np.round(np.mean(spectral_entropy), 4)
            sample_entropy = np.round(np.mean(sample_entropy), 4)
            hjorth_mobility = np.round(np.mean(hjorth_mobility), 4)
            hjorth_complexity = np.round(np.mean(hjorth_complexity), 4)
            nk_fsi = np.round(np.mean(nk_fsi), 4)
            nk_lle = np.round(np.mean(nk_lle), 4)
            nk_sampen = np.round(np.mean(nk_sampen), 4)
            nk_pen = np.round(np.mean(nk_pen), 4)
            nk_lzc = np.round(np.mean(lzc),4)


            #correlation_matrix = np.corrcoef(data[epoch,:,:])
            #upper_triangle_indices = np.triu_indices(len(ch_names), k=1)
            #average_corr = np.mean(correlation_matrix[upper_triangle_indices])
            #average_corr_all.append(average_corr)

            
            df_data = {
                'sub': sub, 
                'day': day, 
                'condition': condition,
                'button': button,
                'jhana': jhana,
                'ch_name': ch_name,
                'n_epochs': n_epochs,
                'n_channels': n_channels,
                
                'lempel_ziv_norm': lempel_ziv_norm,
                'permutation_entropy': permutation_entropy,
                'spectral_entropy': spectral_entropy,
                'sample_entropy': sample_entropy,
                'hjorth_mobility': hjorth_mobility,
                'hjorth_complexity': hjorth_complexity,
                'nk_fsi': nk_fsi,
                'nk_lle': nk_lle,
                'nk_sampen': nk_sampen,
                'nk_pen': nk_pen,
                'nk_lzc': nk_lzc,
                
                'delta': np.round(band_powers['delta'][channel]*10**12,4),
                'theta': np.round(band_powers['theta'][channel]*10**12,4),
                'alpha': np.round(band_powers['alpha'][channel]*10**12,4),
                'beta': np.round(band_powers['beta'][channel]*10**12,4),
                'gamma': np.round(band_powers['gamma'][channel]*10**12,4),
                    }

        
            df_row = pd.DataFrame([df_data])
            
            if first_run:
                df_results = df_row
                first_run = False
            else:
                df_results = pd.concat([df_results, df_row], ignore_index=True)
            df_results.to_csv('eeg_metrics_TMP.csv', index=False)
            

  0%|                                                    | 0/13 [00:58<?, ?it/s]


KeyboardInterrupt: 